In [3]:
import gym
import copy
import numpy as np
import torch
import sys 
sys.path.append("..") 
sys.path.append("../MCTS") 
from monteCarlo import MCTree
import tools
from tools import get_args, registration_envs
registration_envs()

../pretrained_models/PCT_setting1.pt
Loading pre-train upper model ../pretrained_models/PCT_setting1.pt
Pre-train model loaded! ../pretrained_models/PCT_setting1.pt


In [9]:
class usage():
    def __init__(self,K=3):
        self.K=K
        self.box_size_list=[]
        self.simulation_times, self.search_depth, self.rollout_length = (100, None, -1)
        self.mctree=None
        self.new=True
        self.action=None
        
        args = get_args()
        self.env = gym.make(args.id,
                           setting = args.setting,
                           item_set = args.item_size_set,
                           container_size=args.container_size,
                           data_name=args.dataset_path,
                           load_test_data=args.load_dataset,
                           internal_node_holder = args.internal_node_holder,
                           leaf_node_holder = args.leaf_node_holder,
                           LNES = args.lnes,
                           shuffle=args.shuffle,
                           sample_from_distribution=args.sample_from_distribution,
                           sample_left_bound=args.sample_left_bound,
                           sample_right_bound=args.sample_right_bound
                           )
        if args.no_cuda:
            self.device = torch.device('cpu')
        else:
            self.device = torch.device('cuda', args.device)
            torch.cuda.set_device(self.device)
        self.reset()
    def reset(self):
        self.box_size_list=[]
        self.mctree=None
        self.new=True
        self.action=None
        self.obs=self.env.reset()
        self.env.box_creator.preview(1000)
        
    def go(self,box):
        if len(self.box_size_list)<self.K-1: #1~K-1
            self.env.box_creator.box_list[len(self.box_size_list)]=box
            self.box_size_list.append(box)
            return False,None,None
        else: # K
            if self.new:
                self.env.box_creator.box_list[self.K-1]=box
                self.box_size_list.append(box)
                self.obs=self.env.cur_observation()
                self.mctree = MCTree(copy.deepcopy(self.env), copy.deepcopy(self.obs), self.box_size_list, search_depth=self.search_depth, rollout_length=self.rollout_length)
                self.new=False
            else:
                self.env.box_creator.box_list[self.K-1]=box
                self.box_size_list.pop(0)
                self.box_size_list.append(box)
                self.obs=self.env.cur_observation()
                self.mctree = MCTree(copy.deepcopy(self.env), copy.deepcopy(self.obs), self.box_size_list, search_depth=self.search_depth, rollout_length=self.rollout_length)
#                 self.env.box_creator.box_list[self.K-1]=box
#                 self.obs=self.env.cur_observation()
#                 self.mctree.succeed(self.action, box, self.obs)
            
            pl = self.mctree.get_policy(self.simulation_times, zeta=1e-5)
            self.action = self.mctree.sample_action(pl)
            
            # update env
            assert self.env.next_box == self.box_size_list[0]
            obs, r, done, dt = self.env.step(self.action, 'testMCTS')

            if done:
                self.reset()
                return True,self.action,[dt['ratio'], dt['counter'], dt['reward']]
            else:
                return False,self.action,None

# Generate boxes sequence

In [10]:
import pickle
boxes = []
for _ in range(10000):
    box=(np.random.randint(1,6),np.random.randint(1,6),np.random.randint(1,6))
    boxes.append(box)
with open('box_seq.plk','wb') as f:
    pickle.dump(boxes,f)
    f.close()

In [5]:
import pickle
boxes = []
with open('box_seq.plk','rb') as f:
    boxes = pickle.load(f)
    f.close()

# experiment

In [13]:
agent = usage()
results = []
for no, box in enumerate(boxes):
    is_done,action,result=agent.go(box)
    if is_done:
        print(result)
        results.append(result)
        if len(results)>=90:
            break

/root/anaconda3/envs/intelcup/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


cost time 7.3732519999999795
terminated node: 0
reached depth: 2
cost time 6.943910999999957
terminated node: 0
reached depth: 2
cost time 8.133835999999974
terminated node: 0
reached depth: 2
cost time 8.738360000000057
terminated node: 0
reached depth: 2
cost time 9.517659999999978
terminated node: 0
reached depth: 2
cost time 7.86582599999997
terminated node: 0
reached depth: 2
cost time 11.161833999999999
terminated node: 0
reached depth: 2
cost time 8.68689100000006
terminated node: 0
reached depth: 2
cost time 7.06840800000009
terminated node: 0
reached depth: 2
cost time 8.67530899999997
terminated node: 0
reached depth: 2
cost time 8.774609999999939
terminated node: 0
reached depth: 2
cost time 6.161640999999918
terminated node: 0
reached depth: 2
cost time 8.0379089999999
terminated node: 0
reached depth: 2
cost time 9.363652000000002
terminated node: 0
reached depth: 2
cost time 6.18990400000007
terminated node: 0
reached depth: 2
cost time 7.302022999999963
terminated node: 

cost time 6.579688999999689
terminated node: 0
reached depth: 2
cost time 7.872987999999623
terminated node: 0
reached depth: 2
cost time 9.53018800000018
terminated node: 0
reached depth: 2
cost time 7.846262000000024
terminated node: 0
reached depth: 2
cost time 7.809885000000122
terminated node: 0
reached depth: 2
cost time 10.722484999999779
terminated node: 0
reached depth: 2
cost time 10.002163999999993
terminated node: 0
reached depth: 2
cost time 9.849275000000034
terminated node: 0
reached depth: 2
cost time 10.482113000000027
terminated node: 0
reached depth: 2
cost time 8.328913999999713
terminated node: 0
reached depth: 2
cost time 10.31456500000013
terminated node: 0
reached depth: 2
cost time 9.775033999999778
terminated node: 0
reached depth: 2
cost time 11.79361199999994
terminated node: 0
reached depth: 2
cost time 10.54583700000012
terminated node: 0
reached depth: 2
cost time 9.275650999999925
terminated node: 0
reached depth: 2
cost time 9.496965000000273
terminated

cost time 9.69360800000004
terminated node: 0
reached depth: 2
cost time 8.110382999999729
terminated node: 0
reached depth: 2
cost time 7.944696000000022
terminated node: 0
reached depth: 2
cost time 10.84874899999977
terminated node: 0
reached depth: 2
cost time 11.34301400000004
terminated node: 0
reached depth: 2
cost time 10.285397999999986
terminated node: 0
reached depth: 2
cost time 10.3622620000001
terminated node: 0
reached depth: 2
cost time 10.09812200000033
terminated node: 0
reached depth: 2
cost time 9.818395000000237
terminated node: 0
reached depth: 2
cost time 9.888814000000366
terminated node: 0
reached depth: 2
cost time 10.174464000000171
terminated node: 0
reached depth: 2
cost time 11.580228000000261
terminated node: 0
reached depth: 2
cost time 10.463518000000022
terminated node: 0
reached depth: 2
cost time 9.072771999999986
terminated node: 0
reached depth: 2
cost time 7.132760999999846
terminated node: 0
reached depth: 2
cost time 10.906706999999642
terminate

cost time 4.323292000000038
terminated node: 399
reached depth: 0
[0.745, 36, 7.45]
cost time 7.274271000000226
terminated node: 0
reached depth: 2
cost time 8.081050999999206
terminated node: 0
reached depth: 2
cost time 10.360971000000063
terminated node: 0
reached depth: 2
cost time 8.860553999999865
terminated node: 0
reached depth: 2
cost time 8.998596000000362
terminated node: 0
reached depth: 2
cost time 9.030078000000685
terminated node: 0
reached depth: 2
cost time 10.585203000000547
terminated node: 0
reached depth: 2
cost time 8.91459500000019
terminated node: 0
reached depth: 2
cost time 6.97820000000047
terminated node: 0
reached depth: 2
cost time 8.984459000000243
terminated node: 0
reached depth: 2
cost time 9.64526399999977
terminated node: 0
reached depth: 2
cost time 10.776718999999503
terminated node: 0
reached depth: 2
cost time 7.779121999999916
terminated node: 0
reached depth: 2
cost time 9.947756000000481
terminated node: 0
reached depth: 2
cost time 7.60908599

KeyboardInterrupt: 

In [ ]:
#3000 #90个容器的空间利用率

In [21]:
print(results)
mean = np.array(results).mean(axis=0)
print(mean)

[[0.824, 30, 8.24], [0.823, 34, 8.23], [0.806, 28, 8.06], [0.812, 39, 8.120000000000001], [0.814, 29, 8.139999999999999], [0.697, 33, 6.97], [0.789, 34, 7.890000000000001], [0.81, 29, 8.100000000000001], [0.766, 30, 7.66], [0.865, 37, 8.65], [0.771, 23, 7.71], [0.745, 36, 7.45], [0.85, 35, 8.5], [0.829, 30, 8.29], [0.634, 30, 6.34], [0.79, 26, 7.9]]
[ 0.7890625 31.4375     7.890625 ]
